In [1]:
import pandas as pd 
import altair as alt

In [ ]:
success_table = pd.read_csv("GS_exomiser_fig5_input.tsv", sep='\t')
mapping = pd.read_csv('../GS_ID_mapping.csv')


In [ ]:
all_results =pd.read_csv('../new_new_supp_fig_14/GS_exomiser_all_results_optimized.tsv', sep='\t')
filtered_results = all_results[(all_results['P-VALUE'] <= 0.3) & (all_results['#RANK'] <= 30)]
ids = [x.split('_')[0] for x in set(filtered_results['Patient_ID'])]
print(len(ids))

all_genes = []
all_genes_ranks=[]
for ID in set(filtered_results['Patient_ID']):
    patient_results = filtered_results[filtered_results['Patient_ID']==ID]
    unique_genes = list(set(patient_results['GENE_SYMBOL']))
    all_genes += unique_genes
    all_genes_ranks.append(pd.DataFrame(list(zip(list(patient_results['GENE_SYMBOL']), list(patient_results['#RANK'])))).drop_duplicates())
all_genes_ranks_df= pd.concat(all_genes_ranks)
all_genes_ranks_df.columns=['Gene', 'Rank']

In [ ]:
udn_diagnoses = pd.read_csv('../../UDN_2024/5.3.24_update/753_UDNDx_Patients_8.26.24.csv')
udn_diagnoses['Genes'] = udn_diagnoses['Genes'].str.split('; ')
udn_diagnoses = udn_diagnoses.explode('Genes')
all_udn_diagnostic_genes = list(udn_diagnoses['Genes'])

In [ ]:
omim_genes = pd.read_csv('omim_3_genes.tsv')
omim_gene_list = list(omim_genes['0'])

In [ ]:
avg_ranks = []
print(len(all_genes))
print(len(set(all_genes)))
from statistics import mean
import numpy as np
data = []
diagnostic_genes = [x.split('_')[1] for x in list(success_table['ID'])]

for gene in set(all_genes):
    if gene in omim_gene_list:
        gene_2 = '*'+str(gene) 
    else:
        gene_2 = gene
    if gene in diagnostic_genes:
        avg_rank = mean(all_genes_ranks_df[all_genes_ranks_df['Gene']==gene]['Rank'])
        data.append([gene_2, all_genes.count(gene), int(avg_rank), 'benchmarking_diagnostic'])
    elif gene in all_udn_diagnostic_genes:
        avg_rank = mean(all_genes_ranks_df[all_genes_ranks_df['Gene']==gene]['Rank'])
        data.append([gene_2, all_genes.count(gene), int(avg_rank), 'udn_diagnostic'])
    else:
        avg_rank = mean(all_genes_ranks_df[all_genes_ranks_df['Gene']==gene]['Rank'])
        data.append([gene_2, all_genes.count(gene), int(avg_rank), 'not diagnostic'])
df = pd.DataFrame(data, columns=['Gene', 'Count', 'Avg_Rank','Diagnostic'])
df['percent'] = np.array((df['Count']/len(set(ids))) * 100)
print(len(set(ids)))
df =df.sort_values(by='percent', ascending=False)

In [ ]:
plot = df[df['percent']>5]
#plot = plot[plot['percent'] > 5]
print(len(plot))
plot=plot.sort_values(by=['percent', 'Avg_Rank'], ascending=[False,True])
order = list(plot['Gene'])
# plot = df.head(36)
base = alt.Chart(plot)
dots = base.mark_point().mark_point(filled=True, stroke='black',opacity=1).encode(
    x=alt.X('Gene:N', sort=order,axis=alt.Axis(labelAngle=-60)),
    y=alt.Y('percent:Q', title='Percent of Individuals'),
    tooltip =['Gene:N', 'Count:Q', 'Avg_Rank:Q', 'percent'],
    shape=alt.Shape('Diagnostic:N',scale=alt.Scale(range=['square', 'circle', 'diamond']), title='Diagnostic in truth set',legend=alt.Legend(
       orient='top',
       direction='horizontal',
       titleAnchor='middle')),
    color=alt.Color('Avg_Rank:Q', bin=alt.Bin(nice=True, step=5,anchor=30),scale=alt.Scale(scheme='category10'),legend=alt.Legend(
       orient='top',
       direction='horizontal',
       titleAnchor='middle')),
        size = alt.value(80),
)

text = base.mark_text(
    align='right',
    baseline='line-bottom',
    dx=5,
    dy=-5,
    color='black',
    size=11
).encode(
    y=alt.Y('percent', title=None).axis(offset=5, domainOpacity=0),
    x=alt.X('Gene:N', sort=order), 
    text='Avg_Rank:Q'
)

figure_5 = alt.layer(dots).resolve_scale(color='independent').properties(height=150, width=1100)
   

In [ ]:
figure_5.configure_legend(labelLimit=0, labelFontSize=15, labelFont='arial').configure_axis(
    labelFontSize=13,
    titleFontSize=15,
    labelFont='arial').configure_scale(
    bandPaddingInner=0.9

)
